In [14]:
import numpy as np
from matplotlib import pyplot as plt, animation

from sklearn.datasets import make_blobs

from skactiveml.utils import MISSING_LABEL, labeled_indices, unlabeled_indices
from skactiveml.visualization import plot_utilities, plot_decision_boundary
from skactiveml.visualization._misc import mesh
from skactiveml.base import *
  
from skactiveml.classifier import ParzenWindowClassifier
from skactiveml.base import SingleAnnotatorPoolQueryStrategy

from sklearn.metrics import pairwise_distances

In [15]:
class CoreSet(SingleAnnotatorPoolQueryStrategy):
    """ Core Set Selection

    This class implement various core-set based query strategies, i.e., the
    standard greedy algorithm for k-center problem [1], the robust k-center
    algorithm [1].

    Parameters
    ----------
    method: {'greedy', 'robust'}, default='greedy'
        The method to solve the k-center problem, k-center-greedy and robust
        k-center are possible
    missing_label: scalar or string or np.nan or None, default=np.nan
        Value to represent a missing label
    random_state: int or np.random.RandomState
        The random state to use

    References
    ----------
    [1] O. Sener und S. Savarese, „ACTIVE LEARNING FOR CONVOLUTIONAL NEURAL 
    NETWORKS: A CORE-SET APPROACH“, 2018.
    """

    def __init__(
            self, method='greedy', missing_label=MISSING_LABEL, random_state=None
    ):
        super().__init__(
            missing_label=missing_label, random_state=random_state
        )

        self.method = method

    def query(
            self,
            X,
            y,
            candidates=None,
            batch_size=1,
            return_utilities=False,
            **kwargs
    ):

        """ Query the next instances to be labeled

         Parameters
         ----------
         **kwargs
         X: array-like of shape (n_samples, n_features)
            Training data set, usually complete, i.e. including the labeled and
            unlabeled samples
         y: array-like of shape (n_samples, )
            Labels of the training data set (possibly including unlabeled ones
            indicated by self.missing_label)
         candidates: None or array-like of shape (n_candidates), dtype = int or
            array-like of shape (n_candidates, n_features),
            optional (default=None)
            If candidates is None, the unlabeled samples from (X,y) are considered
            as candidates
            If candidates is of shape (n_candidates) and of type int,
            candidates is considered as a list of the indices of the samples in (X,y).
            If candidates is of shape (n_candidates, n_features), the candidates are
            directly given in the input candidates (not necessarily contained in X)
         batch_size: int, optional(default=1)
            The number of samples to be selects in one AL cycle.
         return_utilities: bool, optional(default=False)
            If True, also return the utilities based on the query strategy

         Returns
         ----------
         query_indices: numpy.ndarry of shape (batch_size)
            The query_indices indicate for which candidate sample a label is
            to queried, e.g., `query_indices[0]` indicates the first selected
            sample.
            If candidates in None or of shape (n_candidates), the indexing
            refers to samples in X.
            If candidates is of shape (n_candidates, n_features), the indexing
            refers to samples in candidates.
         utilities: numpy.ndarray of shape (batch_size, n_samples) or
            numpy.ndarray of shape (batch_size, n_candidates)
            The utilities of samples for selecting each sample of the batch.
            Here, utilities means the distance between each data point and its nearest center.
            If candidates is None or of shape (n_candidates), the indexing
            refers to samples in X.
            If candidates is of shape (n_candidates, n_features), the indexing
            refers to samples in candidates.
         """

        X, y, candidates, batch_size, return_utilities = self._validate_data(
            X, y, candidates, batch_size, return_utilities, reset=True
        )

        X_cand, mapping = self._transform_candidates(candidates, X, y)

        if self.method == 'greedy':
            if mapping is not None:
                query_indices, utilities = k_greedy_center(X, y, batch_size, self.random_state_, self.missing_label, mapping)
            else:
                selected_samples = labeled_indices(y=y, missing_label=self.missing_label)
                X_with_cand = np.concatenate((X_cand, X[selected_samples]), axis=0)
                n_new_cand = X_cand.shape[0]
                y_cand = np.full(shape=n_new_cand, fill_value=self.missing_label)
                y_with_cand = np.concatenate((y_cand, y[selected_samples]), axis=None)
                mapping = np.arange(n_new_cand)
                query_indices, utilities = k_greedy_center(X_with_cand, y_with_cand, batch_size, self.random_state_, self.missing_label, mapping, n_new_cand)

        if return_utilities:
            return query_indices, utilities
        else:
            return query_indices

def k_greedy_center(X, y, batch_size, random_state, missing_label=np.nan, mapping=None, n_new_cand=None):
    """
     An active learning method that greedily forms a batch to minimize
     the maximum distance to a cluster center among all unlabeled
     datapoints.
     This method is a static method.

     Parameters:
     ----------
     X: array-like of shape (n_samples, n_features)
        Training data set, usually complete, i.e. including the labeled and
        unlabeled samples
     selected_samples: np.ndarray of shape (n_selected_samples, )
        index of datapoints already selects
     batch_size: int, optional(default=1)
        The number of samples to be selected in one AL cycle.
     random_state: np.random.RandomState
        The random state to use
     missing_label: scalar or string or np.nan or None, default=np.nan
        Value to represent a missing label
     mapping: np.ndarray of shape (n_candidates, ) default None
        Index array that maps `candidates` to `X`.
        (`candidates = X[mapping]`)
     n_new_cand: int or None, default None
        The number of new candidates that are additionally added to X.
        Only used for the case, that in the query function with the
        shape of candidates is (n_candidates, n_feature)
        
     Return:
     ----------
     query_indices: numpy.ndarry of shape (batch_size, )
         The query_indices indicate for which candidate sample a label is
         to queried from the candidates.
         If candidates in None or of shape (n_candidates), the indexing
         refers to samples in X.
         If candidates is of shape (n_candidates, n_features), the indexing
         refers to samples in candidates.
     utilities: numpy.ndarray of shape (batch_size, n_samples) or
         numpy.ndarry of shape (batch_size, n_new_cand)
         The distance between each data point and its nearest center that used
         for selecting the next sample.
         If candidates is None or of shape (n_candidates), the indexing
         refers to samples in X.
         If candidates is of shape (n_candidates, n_features), the indexing
         refers to samples in candidates.
        """
    # read the labeled aka selected samples from the y vector
    selected_samples = labeled_indices(y, missing_label=missing_label)
    
    if mapping is None:
        mapping = unlabeled_indices(y, missing_label=missing_label)
    # initialize the utilities matrix with
    if n_new_cand is None:
        utilities = np.empty(shape=(batch_size, X.shape[0]))
    else:
        utilities = np.empty(shape=(batch_size, n_new_cand))

    query_indices = np.array([], dtype=int)

    for i in range(batch_size):
        if n_new_cand is None:
            utilities[i] = update_distances(X, selected_samples, mapping)
        else:
            update_dist = update_distances(X, selected_samples, mapping)
            utilities[i] = update_dist[mapping]

        # select index
        idx = np.nanargmax(utilities[i])

        if len(selected_samples) == 0:
            idx = random_state.choice(mapping)
            # because np.nanargmax always return the first occurrence is returned

        query_indices = np.append(query_indices, [idx])
        selected_samples = np.append(selected_samples, [idx])

    return query_indices, utilities

def update_distances(X, cluster_centers, mapping):
    """
    Update min distances by given cluster centers.

    Parameters:
    ----------
    X: array-like of shape (n_samples, n_features)
        Training data set, usually complete, i.e. including the labeled and
        unlabeled samples
    cluster_centers: array-like of shape (n_cluster_centers)
        indices of cluster centers
    mapping: np.ndarray of shape (n_candidates, ) default None
        Index array that maps `candidates` to `X`.
        (`candidates = X[mapping]`)

    Return:
    ---------
    result-dist: numpy.ndarray of shape (1, n_samples)
        - if there aren't any cluster centers existed, the default distance
        will be 0
        - if there are some cluster center existed, the return will be the
        distance between each data point and its nearest center after
        each selected sample of the batch. By the case of cluster center the
        value will be np.nan
        - For the indices isn't in mapping, the corresponding value in
        result-dist will be also np.nan
        """
    dist = np.zeros(shape=X.shape[0])

    if len(cluster_centers) > 0:
        cluster_center_feature = X[cluster_centers]
        dist_matrix = pairwise_distances(X, cluster_center_feature)
        dist = np.min(dist_matrix, axis=1)

    result_dist = np.full(X.shape[0], np.nan)
    result_dist[mapping] = dist[mapping]
    result_dist[cluster_centers] = np.nan

    return result_dist


In [16]:
random_state = np.random.RandomState(0)
X, y_true = make_blobs(n_samples=10, n_features=2,
                       centers=[[0, 1], [-3, .5], [-1, -1], [2, 1], [1, -.5]],
                       cluster_std=.7, random_state=random_state)
y_true = y_true % 2
y = np.full(shape=y_true.shape, fill_value=MISSING_LABEL)
y[1] = y_true[1]

In [17]:
qs = CoreSet(method='greedy', random_state=42)

In [18]:
X_cand, _ = make_blobs(n_samples=5, n_features=2,
                       centers=[[0, 1], [-3, .5], [-1, -1], [2, 1], [1, -.5]],
                       cluster_std=.7, random_state=np.random.RandomState(1))
X_with_cand = np.concatenate((X_cand, X), axis=0)
n_new_cand = X_cand.shape[0]
y_cand = np.full(shape=n_new_cand, fill_value=np.nan)
y_with_cand = np.concatenate((y_cand, y), axis=None)
mapping = np.arange(n_new_cand)

In [19]:
cluster_centers = labeled_indices(y=y_with_cand, missing_label=MISSING_LABEL)
cluster_center_feature = X_with_cand[cluster_centers]

In [20]:
dist = pairwise_distances(X_with_cand, cluster_center_feature)
dist = np.min(dist, axis=1)
dist

array([3.2937439 , 3.28753959, 4.93886912, 2.04735565, 5.29096136,
       3.48856278, 0.        , 3.00261471, 2.36945016, 3.72149272,
       3.72218528, 3.63751598, 2.10354519, 1.40087956, 3.7051794 ])

In [21]:
query_idx, utilites = qs.query(X , y, candidates=[0,1,2,3,4], batch_size=2, return_utilities=True)
print(query_idx)
print(utilites)

[4 0]
[[3.48856278        nan 3.00261471 2.36945016 3.72149272        nan
         nan        nan        nan        nan]
 [3.48856278        nan 1.48420062 2.36945016        nan        nan
         nan        nan        nan        nan]]


In [22]:
y_cand = np.full(shape=n_new_cand, fill_value=MISSING_LABEL)
y_with_cand = np.concatenate((y_cand, y), axis=None)
print(y_with_cand)

[nan nan nan nan nan nan  0. nan nan nan nan nan nan nan nan]


In [24]:
query_idx, utilites = qs.query(X , y, candidates=X_cand, batch_size=2, return_utilities=True)
print(utilites)

[[3.2937439  3.28753959 4.93886912 2.04735565 5.29096136]
 [3.2937439  2.52320104 3.79779264 2.04735565        nan]]
